**Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.**

Design a pipeline that includes the following steps:
- Use an automated feature selection method to identify the important features in the dataset.
- Create a numerical pipeline that includes the following steps:
- Impute the missing values in the numerical columns using the mean of the column values.
- Scale the numerical columns using standardisation.
- Create a categorical pipeline that includes the following steps:
- Impute the missing values in the categorical columns using the most frequent value of the column.
- One-hot encode the categorical columns.
- Combine the numerical and categorical pipelines using a ColumnTransformer.
- Use a Random Forest Classifier to build the final model.
- Evaluate the accuracy of the model on the test dataset.


```
# Important Libraries
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
```

```
# Define numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())                 # Scale numerical features
])
```

```
# Define categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent value
    ('onehot', OneHotEncoder())                           # One-hot encode categorical features
])
```

```
# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),       # Apply numerical pipeline to numerical features
    ('cat', categorical_pipeline, categorical_features)   # Apply categorical pipeline to categorical features
])
```

```
# Define feature selection

feature_selection = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))

# Define final pipeline with feature selection and classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),          # Preprocess numerical and categorical features
    ('feature_selection', feature_selection),# Select important features
    ('classifier', RandomForestClassifier()) # Random Forest Classifier
])

# Fit pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)
```

```
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
```

Explanation:

1. Numerical Pipeline: This pipeline handles numerical features. It imputes missing values using the mean of each column and scales the features using standardization.
2. Categorical Pipeline: This pipeline handles categorical features. It imputes missing values using the most frequent value of each column and then performs one-hot encoding.
3. Column Transformer: This combines the numerical and categorical pipelines, applying each to their respective types of features.
4. Feature Selection: Uses a Random Forest Classifier to select important features from the preprocessed data.
5. Final Pipeline: Combines the preprocessor, feature selection, and a Random Forest Classifier.
6. Evaluation: Predicts on the test dataset and evaluates the accuracy of the model.

Possible Improvements:

- Tuning hyperparameters of the Random Forest Classifier for better performance.
- Trying different feature selection methods or feature engineering techniques.
- Handling missing values differently, such as using more advanced imputation methods.
- Exploring other algorithms besides Random Forest to see if they yield better results.

**Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.**

In [17]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [18]:
#load iris dataset
iris = sns.load_dataset('iris')

In [19]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [20]:
from sklearn.preprocessing import LabelEncoder

In [22]:
encoder = LabelEncoder()
iris['species'] = encoder.fit_transform(iris['species'])

In [23]:
iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [24]:
X = iris.drop(labels=['species'],axis=1)
y = iris.species

In [25]:
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [26]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: species, dtype: int64

In [37]:
# split dataset in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

In [38]:
# Define classifiers
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
lr_clf = LogisticRegression(max_iter=1000, random_state=42)

In [39]:
# Define Voting Classifier
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('lr', lr_clf)],
    voting='hard'  # Hard voting combines the predictions by a majority vote
)

In [40]:
# Build the pipeline
pipeline = Pipeline([('clf', voting_clf)])

In [41]:
# Train the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('clf',
                 VotingClassifier(estimators=[('rf',
                                               RandomForestClassifier(random_state=42)),
                                              ('lr',
                                               LogisticRegression(max_iter=1000,
                                                                  random_state=42))]))])

In [42]:
# Train the voting classifier
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))])

In [43]:
# Evaluate accuracy of pipeline
pipeline_acc = accuracy_score(y_test, pipeline.predict(X_test))

# Evaluate accuracy of voting classifier
voting_acc = accuracy_score(y_test, voting_clf.predict(X_test))

print("Accuracy of Pipeline:", pipeline_acc)
print("Accuracy of Voting Classifier:", voting_acc)

Accuracy of Pipeline: 1.0
Accuracy of Voting Classifier: 1.0
